# Ülesanne

Hakkasid tööle uues ettevõttes andmeanalüütikuna. Sinu esimene töö saab olema ettevõtte pesonaliga seotud andmete kontrollimine ja puhastamine.

Väidetavalt on personaliosakond senini kasutatud csv faili, et töötajate kohta käivad andmed kokku koondada. See sama csv fail antakse sulle ülevaatamiseks ning puhastamiseks.

Sinu ülesanne on kasutada oma Pandas'e oskuseid, et tabel korrektselt vormistada.

Eeldused:
- Iga punkti juures tuleb andmetabel eelmisest punktis saadu põhjal kopeerida. Muutuja nimed on etteantud. See on testimise seisukohtalt oluline, et kasutatakse etteantud muutuja nimesid - tester otsib vastavate muutujate väärtuseid notebook'is.

- Andmed tuleb laadida enda Drive'ist näiteks asukohaga:`'/content/drive/MyDrive/google_colab/dirty_dataset.csv'`, kui sa tegid enda Drive'i "google_colab" kausta ning lisasid csv tabeli sinna kausta.

- **Ülesande Notebook'i nimi peab olema 'EX01.ipynb'**
- **See Notebook on *readonly* õigustes, tee sellest endale koopia File --> Save a copy in Drive**

# Impordi vajalikud teegid


In [ ]:
import pandas as pd
import numpy as np
import sys
import os

# Optional: list Drive folder when running in Colab (avoid shell magic for tests)
if "google.colab" in sys.modules:
    drive_dir = "/content/drive/MyDrive/google_colab"
    if os.path.exists(drive_dir):
        print("Files in", drive_dir)
        for name in os.listdir(drive_dir):
            print("-", name)


## 1\. Andmete laadimine drive'ist

- Mount drive
- Lae andmed dirty_dataset.csv failist Colab-i.
- Selleks luua oma Google Drive-i kaust (näites on kausta nimi 'google_colab') ja tõsta csv sinna kausta.
- Kirjuta kausta asukoht (path) allpool muutujasse 'path'.
- Järgmise koodilõugu käivitamisel küsitakse luba, et sinu drive kausta pääseda. See luba tuleb anda ühe korra aktiivse sessiooni jooksul.

In [ ]:
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    path = '/content/drive/MyDrive/google_colab/dirty_dataset.csv'
else:
    path = 'dirty_dataset.csv'

Loe andmed .csv failist DataFrame-i.

In [ ]:
df = pd.read_csv(path, sep=";")

## 2\. Esmane ülevaade
- Uuri andmeid
- Mis tüüpi andmed veergudes on?
- Kas on puuduvaid andmeid?
- Kas on duplikaate?
- Kas on ebaloogilisi väärtuseid?
- Mis on veergude nimed ja kas need tunduvad mõistlikud?

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.describe()

In [ ]:
df.value_counts()

## 3\. Korrasta veerunimed
- Tee originaal DataFrame'ist koopia muutujasse `df_col_rename`
- Nimeta segased veergude nimed ümber
- `Tööala` võiks olla `Amet`
- `Liitumise_Kuupäev` võiks olla `Liitumise_kuupäev`
- Lisa vahetulemus muutujasse `df_col_rename`

Peale muudatusi oleks veergude nimed järgnevad:
![03_veergude_nimetamine.png](https://cs.taltech.ee/services/forge/maksim.tsopov/itx0020-images/raw/branch/main/ex01_pandas/03_veergude_nimetamine.png)

In [ ]:
df_col_rename = df.copy()
df_col_rename = df_col_rename.rename(
    columns={"Tööala": "Amet", "Liitumise_Kuupäev": "Liitumise_kuupäev"}
)

# test
assert "Amet" in df_col_rename.columns
assert "Liitumise_kuupäev" in df_col_rename.columns


## 4\. Korrasta ridadel olevad nimed
- Loo uued veerud: eesnimi veergu "Eesnimi" ja perekonnanimi veergu "Perekonnanimi".
- Lisa vahetulemus muutujasse `df_name`
- Tulemus võiks olla järgnev

![04_nimede_lahutamine.png](https://cs.taltech.ee/services/forge/maksim.tsopov/itx0020-images/raw/branch/main/ex01_pandas/04_nimede_lahutamine.png)

In [ ]:
df_name = df_col_rename.copy()

clean_name = (
    df_name["Nimi1"]
    .astype(str)
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)
    .str.title()
)

name_parts = clean_name.str.split(" ", n=1, expand=True)

df_name["Eesnimi"] = name_parts[0]
df_name["Perekonnanimi"] = name_parts[1]

# test
assert {"Eesnimi", "Perekonnanimi"}.issubset(df_name.columns)
assert df_name["Eesnimi"].notna().all()


## 5\. Korrasta vanus
- Kasuta imputeerimist
- Veendu, et ebaloogilised vanuse read saavad välja filtreeritud
- Veendu, et vanus oleks täisarv
- Lisa vahetulemus muutujasse `df_age`
- Tulemus võiks olla järgnev

![05_nimed.png](https://cs.taltech.ee/services/forge/maksim.tsopov/itx0020-images/raw/branch/main/ex01_pandas/new/5_age.png)

In [ ]:
df_age = df_name.copy()

age_numeric = pd.to_numeric(df_age["Vanus"], errors="coerce")

# Drop rows with invalid ages, keep missing for imputation
valid_mask = age_numeric.between(0, 120) | age_numeric.isna()
df_age = df_age[valid_mask].copy()
age_numeric = age_numeric[valid_mask]

median_age = age_numeric.median()

df_age["Vanus"] = age_numeric.fillna(median_age).round().astype(int)

# test
assert df_age["Vanus"].between(0, 120).all()
assert df_age["Vanus"].dtype.kind in "iu"


## 6\. Korrasta kuupäevad
- Korrasta kuupäevade str formaadid
- Teisenda datetime objektideks
- Lisa vahetulemus muutujasse `df_date`
- Tulemus võiks olla järgnev

![06_kuupäev.png](https://cs.taltech.ee/services/forge/maksim.tsopov/itx0020-images/raw/branch/main/ex01_pandas/new/6_kuupaev.png)

In [ ]:
df_date = df_age.copy()

date_str = df_date["Liitumise_kuupäev"].astype(str).str.replace(r"[./]", "-", regex=True)
date_parsed = pd.to_datetime(date_str, errors="coerce")

median_date = date_parsed.dropna().median()

df_date["Liitumise_kuupäev"] = date_parsed.fillna(median_date)

# test
assert pd.api.types.is_datetime64_any_dtype(df_date["Liitumise_kuupäev"])
assert df_date["Liitumise_kuupäev"].notna().all()


## 7\. Korrasta palk
- kasuta imputeerimist
- palk peab olema täisarv
- Lisa vahetulemus muutujasse `df_salary`
- Tulemus võiks olla järgnev

![07_palk.png](https://cs.taltech.ee/services/forge/maksim.tsopov/itx0020-images/raw/branch/main/ex01_pandas/new/7_palk.png)

In [ ]:
df_salary = df_date.copy()

salary = pd.to_numeric(df_salary["Palk"], errors="coerce")
salary = salary.where(salary >= 0)

mean_salary = salary.mean()

df_salary["Palk"] = salary.fillna(mean_salary).round().astype(int)

# test
assert (df_salary["Palk"] >= 0).all()
assert df_salary["Palk"].dtype.kind in "iu"


## 8\. Mis saab parkimisest?

- Kuna puuduvaid andmeid on niivõrd palju, pole mõtet seda veergu alles hoida. See veerg ei anna midagi juurde
- Lisa vahetulemus muutujasse `df_dropped`
- Tulemus võiks olla järgnev

![08_parkimine.png](https://cs.taltech.ee/services/forge/maksim.tsopov/itx0020-images/raw/branch/main/ex01_pandas/new/8_parkimine.png)

In [ ]:
df_dropped = df_salary.copy()
df_dropped = df_dropped.drop(columns=["Parkimine"])

# test
assert "Parkimine" not in df_dropped.columns


## 9\. Korrasta amet/osakond
- Korrasta formaat, lubatud kategoorilised väärtused: FI, OP, HR, IT
- Imputeeri (ainult õppimise mõttes, tegelikkuses uurime pärisel, kas inimene eksisteerib, mis positsioonil ta töötab)
- Lisa vahetulemus muutujasse `df_title`
- Tulemus võiks olla järgnev

![09_osakond.png](https://cs.taltech.ee/services/forge/maksim.tsopov/itx0020-images/raw/branch/main/ex01_pandas/new/9_osakond.png)

In [ ]:
df_title = df_dropped.copy()

amet_clean = (
    df_title["Amet"]
    .astype(str)
    .str.strip()
    .str.lower()
)

amet_map = {
    "finance": "fi",
    "fi": "fi",
    "ops": "op",
    "op": "op",
    "hr": "hr",
    "it": "it",
}

amet_mapped = amet_clean.map(amet_map).str.upper()

mode_amet = amet_mapped.mode().iloc[0] if not amet_mapped.mode().empty else "IT"

allowed = {"FI", "OP", "HR", "IT"}

df_title["Amet"] = amet_mapped.where(amet_mapped.isin(allowed)).fillna(mode_amet)

# test
assert set(df_title["Amet"].unique()).issubset(allowed)


## 10\. Duplikaadid
- Eemalda duplikaadid, eesnime ja perekonnanime alusel
- Jäta esimene kordus alles
- Lisa vahetulemus muutujasse `df_dupl`
- Tulemus võiks olla järgnev

![10_duplikaadid.png](https://cs.taltech.ee/services/forge/maksim.tsopov/itx0020-images/raw/branch/main/ex01_pandas/new/10_duplikaadid.png)

In [ ]:
df_dupl = df_title.copy()
df_dupl = df_dupl.drop_duplicates(subset=["Eesnimi", "Perekonnanimi"], keep="first")

# test
assert df_dupl.duplicated(subset=["Eesnimi", "Perekonnanimi"]).sum() == 0

## 11\. Loo uued veerud

Loo uus veerg "Aastat_liitumisest", kuhu lisad täisaastad alates liitumisest. Siin tuleb kasutada datetime objekte.

Lisaks, palk kategoriseeritakse veergu "Palk_kategooria", kolme klassi vastavalt selle väärtusele:
- "madal" – kui palk on 0 kuni 29 999 (kaasaarvatud 0 ja väiksem kui 30 000)
- "keskmine" – kui palk on 30 000 kuni 59 999
- "kõrge" – kui palk on 60 000 või rohkem

Kõigi muude juhtude puhul (näiteks kui väärtus puudub või pole arv) määratakse kategooriaks NaN

Lisa vahetulemus muutujatesse:
- df_dt  ("Aastat_liitumisest")
- df_cat ("Palk_kategooria")

Tulemus võiks olla järgnev:
![11_palgakategooria.png](https://cs.taltech.ee/services/forge/maksim.tsopov/itx0020-images/raw/branch/main/ex01_pandas/new/11_palgakategooria.png)

In [ ]:
today = pd.Timestamp('today')

df_dt = df_dupl.copy()

# Convert dates for calculation (keep datetime here)
join_dates = pd.to_datetime(df_dt["Liitumise_kuupäev"], errors="coerce")

df_dt["Aastat_liitumisest"] = (
    (today - join_dates).dt.days // 365
).astype(int)

df_dt["Liitumise_kuupäev"] = join_dates

df_cat = df_dt.copy()

conditions = [
    (df_cat["Palk"] >= 0) & (df_cat["Palk"] < 30000),
    (df_cat["Palk"] >= 30000) & (df_cat["Palk"] < 60000),
    (df_cat["Palk"] >= 60000),
]

choices = ["madal", "keskmine", "kõrge"]

palk_cat = pd.Series(
    np.select(conditions, choices, default=None),
    index=df_cat.index,
    dtype="object",
)

df_cat["Palk_kategooria"] = palk_cat

# test
assert (df_dt["Aastat_liitumisest"] >= 0).all()
assert df_cat["Palk_kategooria"].dropna().isin(choices).all()

## 12\. Loo uus tabel ja liida see olemasolevaga kasutades selleks indeksit
- loo uus dataframe (`df_hoone`), kus on kaks veergu
  - Id (algsest tabelist)
  - Hoone [A,B,C], mil me teame et,
  ID ning hoone vahelised seosed on järgmised:

```
    Id  Hoone
0	124	A
1	152	B
2	632	C
3	853	A
4	963	C
5	84	B
6	863	A
7	973	A
8	111	B
9	142	C
```

- See lisainfo määrab töötaja asukoha vastavalt tähistatud hoones.
- Liida see uus hoone tabel olemasoleva personali tabeliga.
- Vahevastus salvesta muutujasse `df_merged`

Tulemus võiks olla järgnev:
![12_hoone.png](https://cs.taltech.ee/services/forge/maksim.tsopov/itx0020-images/raw/branch/main/ex01_pandas/new/12_hoone.png)

In [ ]:
# Eemalda olemasolev indeks ja määra indeks ise
df_reset = df_cat.reset_index(drop=True)

# For merge step, keep date as string (expected in tests)
df_reset["Liitumise_kuupäev"] = df_reset["Liitumise_kuupäev"].dt.strftime("%Y-%m-%d")

# Loome hoone tabeli olemasolevate indeksite põhjal + uus veerg hoonete tähistustega
df_hoone = pd.DataFrame(
    {
        "Id": [124, 152, 632, 853, 963, 84, 863, 973, 111, 142],
        "Hoone": ["A", "B", "C", "A", "C", "B", "A", "A", "B", "C"],
    }
)

df_merged = df_reset.merge(df_hoone, on="Id", how="left")

# test
assert "Hoone" in df_merged.columns

## 13\. Sorteeri ja tegele indeksiga

  - Sorteeri liitumise kuupäeva järgi hilisemast varasemaks
  - Veendu, et indeks oleks kasvav. Esimene rida 0
  - Lisa vahevastus muutjasse `df_sort`

  Tulemus võiks olla järgnev:

  ![13_sorteeri.png](https://cs.taltech.ee/services/forge/maksim.tsopov/itx0020-images/raw/branch/main/ex01_pandas/new/13_sorteeri.png)

In [ ]:
df_sort = df_merged.copy()

# Convert date strings back to datetime for sorting and downstream tests
df_sort["Liitumise_kuupäev"] = pd.to_datetime(
    df_sort["Liitumise_kuupäev"], errors="coerce"
)

df_sort = df_sort.sort_values("Liitumise_kuupäev", ascending=False).reset_index(drop=True)

# test
assert df_sort.index.is_monotonic_increasing


## 14\. Muuda veergude järjekorda selliselt, et Id oleks kõige vasakpoolsem veerg ning sellele järgneks paremal:
- Liitumise_kuupäev
- Aastat_liitumisest
- Amet
- Eesnimi
- Perekonnanimi
- Palk
- Palk_kategooria
- Vanus

Vahevastus pane muutujasse `df_col_sorted`

Tulemus võiks olla järgnev:
![14_veergude_tõstmine.png](https://cs.taltech.ee/services/forge/maksim.tsopov/itx0020-images/raw/branch/main/ex01_pandas/new/14_veergude_tostmine.png)

In [ ]:
df_col_sorted = df_sort.copy()

col_order = [
    "Id",
    "Liitumise_kuupäev",
    "Aastat_liitumisest",
    "Amet",
    "Eesnimi",
    "Perekonnanimi",
    "Palk",
    "Palk_kategooria",
    "Vanus",
]

df_col_sorted = df_col_sorted[col_order]

# test
assert list(df_col_sorted.columns) == col_order


## 15\. Salvesta tabel tagasi csv kujule
- Indeks pole oluline failis
- Veendu, et fail on salvetatud enda drive kausta

In [ ]:
from datetime import datetime
import pytz

# Define the Estonian timezone
est_timezone = pytz.timezone('Europe/Tallinn')

# Get the current time in Estonian timezone and format it to include time
timestamp = datetime.now(est_timezone).strftime("%Y%m%d_%H%M%S")

if "google.colab" in sys.modules:
    path = f"/content/drive/MyDrive/google_colab/clean_dataset_{timestamp}.csv"
else:
    path = f"clean_dataset_{timestamp}.csv"

df_col_sorted.to_csv(path, index=False)